In [2]:
pip install rtree

In [10]:
pip install dask-geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.3 MB/s eta 0:00:00


In [12]:
pip install --upgrade dask-geopandas

In [20]:
import pandas as pd
import geopandas as gpd
import numpy as np
import dask.dataframe as dd
import dask.distributed
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points
from scipy.spatial import cKDTree
import copy

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
pd.set_option('display.max_rows',10)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', 200)

In [5]:
gdf_center = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市　重心/gdf_center.shp')

In [6]:
gdf_railway = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/鉄道/N02-22_RailroadSection.shp', encoding='shift-jis') #このデータは道路中心線

In [205]:
gdf_100m = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市　重心/gdf_osyu_100m_mesh.shp')

In [206]:
gdf_100m.to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/確認/奥州市　鉄道　確認/gdf_100m.csv', encoding='shift-jis')

In [210]:
gdf_railway.to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/確認/奥州市　鉄道　確認/gdf_railway.csv', encoding='shift-jis')

In [209]:
gdf_railway

,N02_001,N02_002,N02_003,N02_004,geometry
0,23,5,沖縄都市モノレール線,沖縄都市モノレール,"LINESTRING (127.67948 26.21454, 127.67970 26.21474, 127.67975 26.21480, 127.68217 26.21728, 127.68357 26.21862, 127.68394 26.21891, 127.68419 26.21905)"
1,12,5,いわて銀河鉄道線,アイジーアールいわて銀河鉄道,"LINESTRING (141.29139 40.33740, 141.29176 40.33723, 141.29243 40.33692, 141.29323 40.33654, 141.29379 40.33624, 141.29411 40.33608, 141.29490 40.33563, 141.29624 40.33477, 141.29813 40.33354, 141...."
2,12,5,いわて銀河鉄道線,アイジーアールいわて銀河鉄道,"LINESTRING (141.27554 40.23936, 141.27567 40.23884, 141.27587 40.23827, 141.27622 40.23756, 141.27656 40.23694, 141.27722 40.23579, 141.27789 40.23462, 141.27856 40.23344, 141.27922 40.23236, 141...."
3,12,5,いわて銀河鉄道線,アイジーアールいわて銀河鉄道,"LINESTRING (141.28659 40.26092, 141.28538 40.25874)"
4,12,5,いわて銀河鉄道線,アイジーアールいわて銀河鉄道,"LINESTRING (141.29082 40.28615, 141.29089 40.28582, 141.29103 40.28485)"
...,...,...,...,...,...
21911,12,5,あいの風とやま鉄道線,あいの風とやま鉄道,"LINESTRING (137.24023 36.71916, 137.24056 36.71999)"
21912,12,5,あいの風とやま鉄道線,あいの風とやま鉄道,"LINESTRING (137.24056 36.71999, 137.24071 36.72035, 137.24116 36.72148, 137.24182 36.72320, 137.24206 36.72381, 137.24235 36.72457, 137.24286 36.72585, 137.24308 36.72641, 137.24318 36.72667, 137...."
21913,11,2,宗谷線,北海道旅客鉄道,"LINESTRING (142.46548 44.33083, 142.46548 44.33151)"
21914,11,2,札沼線,北海道旅客鉄道,"LINESTRING (141.44875 43.18960, 141.44622 43.18756, 141.44481 43.18642, 141.44369 43.18551, 141.44097 43.18335, 141.43831 43.18119, 141.43707 43.18019)"


In [7]:
#座標系を変更
src_proj = 4326 # 変換前の座標系を指定
dst_proj = 6678  # 変換後の座標系を指定
# ポイント（ダムデータ、TransformPointの引数は緯度,経度の順番で指定）
gdf_center.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_center = gdf_center.to_crs(epsg=dst_proj)  # 変換後座標に変換

In [8]:
#座標系を変更
src_proj = 6668 # 変換前の座標系を指定
dst_proj = 6678  # 変換後の座標系を指定
# ポイント（ダムデータ、TransformPointの引数は緯度,経度の順番で指定）
gdf_railway.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_railway = gdf_railway.to_crs(epsg=dst_proj)  # 変換後座標に変換

In [22]:
from tqdm import tqdm

tqdm.pandas()
# Create a spatial index for the lines
sindex = gdf_railway.sindex

In [26]:
# Define a function to get the nearest line for each point and the distance to it using the spatial index
def get_nearest_line_and_distance_with_sindex(point, lines, sindex, buffer=100):
    # Get the bounding box coordinates of the point and expand it by a buffer
    minx, miny, maxx, maxy = point.bounds
    bbox = (minx - buffer, miny - buffer, maxx + buffer, maxy + buffer)

    # Get the possible matches with the spatial index
    possible_matches_index = list(sindex.intersection(bbox))
    possible_matches = lines.iloc[possible_matches_index]

    # Check if there are no matches
    if possible_matches.empty:
        raise ValueError(f"No lines found within buffer for point {point}")

    # Use the unary_union attribute to create a MultiLineString object
    # This allows us to use the nearest_points function
    nearest_line = nearest_points(point, possible_matches.unary_union)[1]
    distance = point.distance(nearest_line)
    return nearest_line, distance

# Apply the function to all the centroids using the spatial index and the expanded bounding box
# Use a progress bar to track the progress
gdf_center[['nearest_line', 'distance']] = gdf_center.progress_apply(lambda row: get_nearest_line_and_distance_with_sindex(row['geometry'], gdf_railway['geometry'], sindex), axis=1, result_type='expand')

gdf_center.head()

  0%|          | 1/101386 [00:04<127:13:00,  4.52s/it]


ValueError: ignored

In [15]:
import dask.dataframe as dd
from rtree import index
from dask.diagnostics import ProgressBar

# Convert the GeoDataFrame to a Dask DataFrame
ddf_center = dd.from_pandas(gdf_center, npartitions=4)  # adjust npartitions according to your CPU cores

# Define a function to get the nearest line for each point and the distance to it using the spatial index
def get_nearest_line_and_distance_with_sindex(df):
    # Create a spatial index for the railway lines in this partition
    idx = index.Index()
    for i, geometry in enumerate(gdf_railway.geometry):
        idx.insert(i, geometry.bounds)

    result = []
    for row in df.itertuples():
        point = row.geometry
        buffer = 10

        # Get the bounding box coordinates of the point and expand it by a buffer
        minx, miny, maxx, maxy = point.bounds
        bbox = (minx - buffer, miny - buffer, maxx + buffer, maxy + buffer)

        # Get the possible matches with the spatial index
        possible_matches_index = list(idx.intersection(bbox))
        possible_matches = gdf_railway.iloc[possible_matches_index]

        # Check if there are no matches
        if possible_matches.empty:
            result.append((None, None))
            continue

        # Use the unary_union attribute to create a MultiLineString object
        # This allows us to use the nearest_points function
        nearest_line = nearest_points(point, possible_matches.unary_union)[1]
        distance = point.distance(nearest_line)
        result.append((nearest_line, distance))
    return pd.DataFrame(result, columns=['nearest_line', 'distance'])

# Apply the function to all the partitions using the map_partitions function
# Use a progress bar to track the progress
with ProgressBar():
    ddf_result = ddf_center.map_partitions(get_nearest_line_and_distance_with_sindex, meta=('object')).compute(scheduler='processes')

ddf_result.head()


[                                        ] | 0% Completed | 707.38 ms


KeyboardInterrupt: ignored

In [13]:
import dask.dataframe as dd
import dask_geopandas as dask_gpd
from dask.diagnostics import ProgressBar
from rtree import index

# Convert the GeoDataFrame to a Dask DataFrame
ddf_center = dask_gpd.from_pandas(gdf_center, npartitions=4)  # adjust npartitions according to your CPU cores

# Define a function to get the nearest line for each point and the distance to it using the spatial index
def get_nearest_line_and_distance_with_sindex(df):
    # Create a spatial index for the railway lines in this partition
    idx = index.Index()
    for i, geometry in enumerate(gdf_railway.geometry):
        idx.insert(i, geometry.bounds)

    result = []
    for row in df.itertuples():
        point = row.geometry
        buffer = 10

        # Get the bounding box coordinates of the point and expand it by a buffer
        minx, miny, maxx, maxy = point.bounds
        bbox = (minx - buffer, miny - buffer, maxx + buffer, maxy + buffer)

        # Get the possible matches with the spatial index
        possible_matches_index = list(idx.intersection(bbox))
        possible_matches = gdf_railway.iloc[possible_matches_index]

        # Check if there are no matches
        if possible_matches.empty:
            result.append((None, None))
            continue

        # Use the unary_union attribute to create a MultiLineString object
        # This allows us to use the nearest_points function
        nearest_line = nearest_points(point, possible_matches.unary_union)[1]
        distance = point.distance(nearest_line)
        result.append((nearest_line, distance))
    return pd.DataFrame(result, columns=['nearest_line', 'distance'])

# Apply the function to all the partitions using the map_partitions function
# Use a progress bar to track the progress
with ProgressBar():
    ddf_result = ddf_center.map_partitions(get_nearest_line_and_distance_with_sindex, meta=('object')).compute(scheduler='processes')

ddf_result.head()


AttributeError: ignored

In [10]:
def get_nearest_line_and_distance(point, lines):
    # Use the unary_union attribute to create a MultiLineString object
    # This allows us to use the nearest_points function
    nearest_line = nearest_points(point, lines.unary_union)[1]
    distance = point.distance(nearest_line)
    return nearest_line, distance

gdf_center[['nearest_line', 'distance']] = gdf_center.apply(lambda row: get_nearest_line_and_distance(row['geometry'], gdf_railway['geometry']), axis=1, result_type='expand')

gdf_center.head()

KeyboardInterrupt: ignored

In [193]:
# ポイントデータのGeodataframe
point_gdf = gdf_center  # ポイントデータのGeodataframeを指定してください

# ラインデータのGeodataframe
line_gdf = gdf_railway  # ラインデータのGeodataframeを指定してください

# ポイントデータの座標をNumPy配列に変換
point_coords = np.array(point_gdf.geometry.apply(lambda geom: (geom.x, geom.y)).tolist())

# ラインデータの座標をNumPy配列に変換
line_coords = np.array(line_gdf.geometry.apply(lambda geom: geom.coords[0]).tolist())

# 最近傍検索のためのKDTreeを構築
kdtree = cKDTree(line_coords)

# 各ポイントデータの最近傍ラインデータのインデックスと距離を求める
distances, line_indices = kdtree.query(point_coords)

# 最近傍ラインデータのGeodataframeを作成
nearest_lines = line_gdf.iloc[line_indices]

# 距離情報を追加
nearest_lines['distance'] = distances

#結合して変わったindexを変換
nearest_lines.reset_index(drop=True, inplace=True)

# #geometryが二つあると出力できないのでcenter_geometryを削除
gdf_center.drop(columns=['longitude', 'latitude', 'geometry'], axis=1, inplace=True)
nearest_lines.drop('geometry', axis=1, inplace=True)

# 結果の表示
gdf_railway = gdf_center.join(nearest_lines)

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
<ipython-input-193-77c4de133f99>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nearest_lines.drop('geometry', axis=1, inplace=True)


In [197]:
#線路のデータフレームのカラム名を変換
gdf_railway.rename(columns={'N02_001': 'Railway_classification', 'N02_002': 'Railway_Business_Type', 'N02_003': 'Railway_route_name', 'N02_004':'Railway_operating_company'}, inplace=True)

In [201]:
gdf_railway.to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市 最近傍鉄道/奥州市_最近傍線路.csv', encoding='shift-jis', index=False)

In [218]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.ops import nearest_points

# Load the data
df_100m = pd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/確認/奥州市　鉄道　確認/gdf_100m.csv', encoding='cp932')
df_railway = pd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/確認/奥州市　鉄道　確認/gdf_railway.csv', encoding='cp932')

# Convert the 'geometry' column to Shapely objects
df_100m['geometry'] = df_100m['geometry'].apply(wkt.loads)
df_railway['geometry'] = df_railway['geometry'].apply(wkt.loads)

# Convert the DataFrames to GeoDataFrames
gdf_100m = gpd.GeoDataFrame(df_100m, geometry='geometry', crs="EPSG:6668")
gdf_railway = gpd.GeoDataFrame(df_railway, geometry='geometry', crs="EPSG:6668")

# Transform the CRS to EPSG:6678
gdf_100m = gdf_100m.to_crs("EPSG:6678")
gdf_railway = gdf_railway.to_crs("EPSG:6678")

# Calculate the centroids of each polygon
gdf_100m['centroid'] = gdf_100m['geometry'].centroid
gdf_100m.set_geometry('centroid', inplace=True)

# Create a spatial index on gdf_railway
spatial_index = gdf_railway.sindex

def find_nearest(row, spatial_index, gdf2):
    # Find the nearest railway line to the centroid
    nearest_index = spatial_index.nearest(row['geometry'].bounds)
    nearest_geom = nearest_points(row['geometry'], gdf2.loc[nearest_index].geometry.unary_union)[1]

    # Find the distance to the nearest railway line
    distance = row['geometry'].distance(nearest_geom)

    return distance

# Calculate the distance from each centroid to the nearest railway line
gdf_100m['nearest_distance'] = gdf_100m.apply(find_nearest, axis=1, args=(spatial_index, gdf_railway))

gdf_100m.head()

TypeError: ignored

In [217]:
print(type(gdf_100m['geometry']))
print(type(gdf_railway.geometry.unary_union))

<class 'geopandas.geoseries.GeoSeries'>
<class 'shapely.geometry.multilinestring.MultiLineString'>
